# Máltækni, verkefni 7

## 1) Levenshtein–fall (2 stig)

Í þessum hluta verkefnisins eigið þið að útfæra ykkar eigið Levenshtein–fall. Þið megið nota hvað sem er ykkur til innblásturs (það er m.a. sauðakóði á Wikipedia) en þið þurfið að útfæra fallið sjálf (með öðrum orðum ekki skila inn import levenshtein-distance). Athugið að þið megið nota pakka á borð við numpy eða hvað sem ykkur dettur í hug ef það gagnast ykkur. Látið fallið reikna út Levenshtein–fjarlægðina á nokkrum strengjum að eigin vali.

Útfærum bæði endurkvæma útgáfu og útgáfu sem notar Wagner–Fischer reikniritið í `levenshtein.py` og með prófum í `tests/test_levenshtein.py`.

Sem hliðarspor þá setti ég upp með `poetry` og prófaði `coverage` pakkann til að fylgjast code coverage, sem við náum í 100%! (Er að nota `tokenizer` úr verkefni 3.)


```bash
> poetry run coverage run -m pytest
=================== test session starts ===================
tests/test_dictionary.py .....                       [ 22%]
tests/test_levenshtein.py ...                        [ 36%]
tests/test_tokenizer.py ..............               [100%]
=================== 22 passed in 0.02s ====================

> poetry run coverage report -m    
Name                        Stmts   Miss  Cover   Missing
---------------------------------------------------------
dictionary.py                  18      0   100%
levenshtein.py                 25      0   100%
tests/__init__.py               0      0   100%
tests/test_dictionary.py       18      0   100%
tests/test_levenshtein.py      10      0   100%
tests/test_tokenizer.py        35      0   100%
tokenizer.py                   37      0   100%
---------------------------------------------------------
TOTAL                         143      0   100%
```

Endurkvæma útgáfan virkar fínt á stutta strengi en um leið og þeir verða átta stafir eða lengri tekur það **langan** tíma að reikna niðurstöðu. Wagner–Fischer er mun fljótari að reikna lengri strengi, þó það taki smá stund.

```python
def ld_recusive(a, b):
    """
    Calculate levenshtein distance between two strings via rescursion
    """
    # we only allow strings
    if not isinstance(a, str) or not isinstance(b, str):
        return 0

    # |a| = 0 => |b|
    if len(a) == 0:
        return len(b)

    # |b| = 0 => |a|
    if len(b) == 0:
        return len(a)

    if a[0] == b[0]:
        return ld_recusive(a[1:], b[1:])

    return 1 + min(
        [
            ld_recusive(a[1:], b),
            ld_recusive(a, b[1:]),
            ld_recusive(a[1:], b[1:]),
        ]
    )
```

In [1]:
from levenshtein import ld_recusive, ld_wf

ld_recusive("hello", "world")

4

```python
def ld_wf(s, t):
    """
    Calculate levenshtein distance between two strings via Wagner–Fischer
    https://en.wikipedia.org/wiki/Wagner%E2%80%93Fischer_algorithm
    """
    if not isinstance(s, str) or not isinstance(t, str):
        return 0

    m = len(s) + 1
    n = len(t) + 1
    d = [[0 for _ in range(n)] for _ in range(m)]

    for i in range(1, m):
        d[i][0] = i

    for j in range(1, n):
        d[0][j] = j

    for j in range(1, n):
        for i in range(1, m):
            substitutionCost = 0 if s[i - 1] == t[j - 1] else 1

            d[i][j] = min(
                [d[i - 1][j] + 1, d[i][j - 1] + 1, d[i - 1][j - 1] + substitutionCost]
            )

    return d[m - 1][n - 1]
```

In [2]:
ld_wf("hello world this is a test string", "and now for something completely different")

29

## 2) Orðabókaruppfletting (2 stig)

Í þessum hluta verkefnisins eigið þið að útfæra orðabókaruppflettingu sem greinir rangt skrifuð orð út frá því hvort þau eru til í orðaforðanum eða ekki. Þetta má útfæra á ýmsa vegu, verið endilega frumleg með gagnastrúktúra. Það sem við viljum fá út úr þessu er einfaldlega fall eða klasi sem kannar hvort eitthvað ákveðið orð sé til í fyrirfram skilgreindum orðaforða. Til þess að skilgreina orðaforðann skuluð þið nota gagnasafnið sem þið söfnuðuð sjálf (með öðrum orðum ætti forritið ykkar að líta svo á að allar stakar orðmyndir sem finnast í gögnunum ykkar séu réttar en allar orðmyndir sem finnast ekki í gögnunum ykkar séu rangar). Sýnið niðurstöðurnar þegar forritið fær orð sem inntak sem er ekki í orðaforðanum (þ.e.a.s. sýnið þegar forritið finnur villu) og þegar forritið fær orð sem inntak sem er til í orðaforðanum (þ.e.a.s. sýnið þegar forritið finnur rétt skrifað orð). Það má nota hvaða forritasafn sem þið viljið í þetta (bara ekki skila import dictionary-lookup).

Byrjum með einfalda orðabók sem tekur við strengjum og segir til um hvort strengurinn sé til eða ekki (nota enga skemmtilega gagnastrúktúra hér, bara einfalda lista).

Notum sönglagatexta og tokenizer úr verkefni 3.

Búum líka til hjálparfall sem tekur við setningu, tókar og skilar hvort orð séu í orðabókinni.

In [3]:
from tokenizer import WORD, tokenize

with open("./data/songlog.txt", "rb") as input_file:
    songlog = input_file.read().decode("utf8")
    tokens = [token[0] for token in tokenize(songlog) if token[1] == WORD]


```python
from tokenizer import tokenize, WORD


class Dictionary:
    def __init__(self, values):
        if not isinstance(values, list):
            raise "values must be list"

        clean = []
        for value in values:
            if not isinstance(value, str):
                raise "non string value in values"
            if value not in clean:
                clean.append(value.lower())

        self.values = clean

    def has(self, value):
        return value.lower() in self.values

    def sentence(self, s):
        tokens = [token[0] for token in tokenize(s) if token[1] == WORD]

        result = [(word, self.has(word)) for word in tokens]

        return result
```

In [4]:
from dictionary import Dictionary

In [5]:
dict = Dictionary(tokens)

In [6]:
dict.has('sólin')

True

In [7]:
dict.has('bull')

False

In [8]:
# Hjálparfall sem tókar og athugar hvert orð
dict.sentence('Ástin er foo, veruleikinn er bar með þér og baz')

[('Ástin', True),
 ('er', True),
 ('foo', False),
 ('veruleikinn', True),
 ('er', True),
 ('bar', True),
 ('með', True),
 ('þér', True),
 ('og', True),
 ('baz', False)]

## 3) Transformer <mask> sem leiðréttingartól (2 stig)

Í þessum hluta verkefnisins eigið þið að kanna hvort þið getið beitt MLM líkani (t.d. BERT eða félögum hans – verið alveg viss um að hann tali rétta tungumálið hins vegar, það er ekki hentugt að beita IceBERT á texta á ensku) til þess að finna villur og leiðrétta þær. Það skal gera á eftirfarandi hátt:

1) Sendið setningu sem inniheldur rangt skrifað orð inn í orðabókaruppflettinguna (þetta má t.a.m. gera með því að tóka setninguna og senda hvern tóka fyrir sig inn í forritið). Þar með ættuð þið að hafa fundið villuna.

2) Skiptið rangt skrifaða orðinu út fyrir <mask> í setningunni. Sendið setninguna síðan í gegnum Transformer líkanið og fáið út þau 10 orð sem líkanið telur líklegast að eigi að standa í stað <mask>.

3) Beitið Levenshtein–fallinu ykkar á þessi 10 orð sem líkanið lagði til (með öðrum orðum: hver er Levenshtein–fjarlægðin á milli upphaflega villuorðsins sem þið földuð og þessara orða). Hvert þeirra hefur lægstu fjarlægðina? Er það gild leiðrétting á orðinu sem þið skrifuðuð rangt til að byrja með (eða ef mörg orð koma til greina, er það meðal þeirra sem hafa lægstu Levenshtein–fjarlægðina)? Athugið að það er ekkert víst að þið fáið rétta niðurstöðu. Ef rétta orðið finnst alls ekki, prófið aðra (einfaldari) setningu ykkur til gamans.

Notum IceBERT og búum til leiðréttingarfall fyrir íslensku!

In [9]:
from transformers import RobertaForMaskedLM, RobertaTokenizer, pipeline

MODEL_NAME = "mideind/IceBERT"

tokenizer = RobertaTokenizer.from_pretrained(MODEL_NAME)
model = RobertaForMaskedLM.from_pretrained(MODEL_NAME)
pipe = pipeline("fill-mask", model=model, tokenizer=tokenizer)

In [10]:
MAX_DISTANCE = 2


def pick_possible(word, possibles):
    """Fyrir gefið `word` velur það sem er líkast og innan hámarks LD"""
    ranked_by_distance = sorted(
        [
            {
                "word": possible["token_str"].strip(),
                "distance": ld_wf(word, possible["token_str"].strip()),
            }
            for possible in possibles
        ],
        key=lambda i: i["distance"],
    )

    if ranked_by_distance[0]["distance"] <= MAX_DISTANCE:
        return ranked_by_distance[0]["word"]

    return word


def correct(sentence, dictionary, pipeline):
    """Athugar orð í gefinni setningu og reynir að leiðrétta."""
    checked = dict.sentence(sentence)

    # Ef engar villur, skilum setningu
    # Hér erum við búin að strípa út allt sem er ekki WORD en þetta er einföldun
    if all([c[1] for c in checked]):
        return "".join([s[0] for s in sentence])

    corrected = []
    index = -1

    # Förum gegnum öll orð sem eru ekki í orðabók og reynum hvert
    for c in checked:
        index = index + 1
        if c[1]:
            corrected.append(c[0])
            continue

        # Útbúum maskaða setningu úr því sem komið er og því sem fylgir
        masked = " ".join(
            corrected
            + ["<mask>"]
            + [c[0] for c in checked[index + 1 : index + len(checked)]]
        )
        possibles = pipe(masked)

        # Veljum besta orð úr möguleikum eða í versta fallið vitlausa orðið
        corrected.append(pick_possible(c[0], possibles))

    return " ".join(corrected)

In [11]:
# Skilar réttri setningu óbreyttri
correct("Halló! Hvað segið þið gott?", dict, pipe)

'Halló! Hvað segið þið gott?'

In [18]:
# náum að laga innsláttarvillu, en strípum öll greinamerki
correct("Í dag, er góðr... dagur!", dict, pipe)

'Í dag er góður dagur'

In [13]:
# og aðra
correct("Biggjum okkur hús", dict, pipe)

'byggjum okkur hús'

In [14]:
# og nær að leiðrétta tvær villur!
correct("hva er í matin", dict, pipe)

'Hvað er í matinn'

In [15]:
# setningar sem eru óljósar er erfitt að leiðrétta
# lækkaði mörkin töluvert niður (í 2) eftir að hafa fengi mun meira af false-positives
correct("hún er fe", dict, pipe)

'hún er ég'

In [16]:
# og bara verður betra með lengri setningum
correct("likillinn að húsinu er týndur fra því í gaer þegar við forum út í biil með hundin", dict, pipe)

'lykillinn að húsinu er týndur frá því í gær þegar við fórum út í bíl með hundinn'